# Étape 5 — Intégration ML + OR-Tools

Ce notebook intègre le modèle ML entraîné à l'étape 4 dans l'optimisation des routes avec OR-Tools.

## Objectif

Utiliser les prédictions ML (temps de trajet prédit) au lieu de la distance brute pour optimiser les routes, permettant de :
- Éviter les chemins congestionnés
- Prendre en compte les obstacles
- Optimiser le temps réel plutôt que la distance

## Étapes

1. Charger le modèle ML et les données de l'entrepôt
2. Construire une nouvelle matrice de coûts basée sur `predicted_time` au lieu de `distance`
3. Relancer l'optimisation OR-Tools avec cette nouvelle matrice
4. Comparer les routes avec et sans ML
5. Analyser les métriques et les différences


In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import os
import joblib
import json

# Add src directory to path
sys.path.insert(0, os.path.join('..', 'src'))

from optimize_routes import RouteOptimizer, load_warehouse_data
from utils import euclidean_distance

# Configuration
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('seaborn')
sns.set_palette("husl")
np.random.seed(42)

print("Imports réussis")


## 1. Chargement des Données et du Modèle ML

Chargement :
- Données de l'entrepôt (points, labels)
- Matrice de distances originale
- Modèle ML entraîné
- Métriques du modèle


In [ ]:
# Charger les données de l'entrepôt
print("=" * 60)
print("CHARGEMENT DES DONNÉES")
print("=" * 60)

distance_matrix, points, point_labels = load_warehouse_data(data_dir="../data/raw")
print(f"Points chargés : {len(points)}")
print(f"Matrice de distances : {distance_matrix.shape}")

# Charger le modèle ML
print("\n" + "=" * 60)
print("CHARGEMENT DU MODÈLE ML")
print("=" * 60)

metrics_path = "../data/processed/ml_model_metrics.json"
with open(metrics_path, 'r') as f:
    ml_metrics = json.load(f)

model_type = ml_metrics['model']
print(f"Modèle sélectionné : {model_type}")

if model_type == "Random Forest":
    model_path = "../data/processed/ml_model_rf.pkl"
    ml_model = joblib.load(model_path)
    scaler = None
    print(f"Modèle Random Forest chargé : {model_path}")
else:
    model_path = "../data/processed/ml_model_mlp.pkl"
    scaler_path = "../data/processed/ml_scaler.pkl"
    ml_model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print(f"Modèle MLP chargé : {model_path}")
    print(f"Scaler chargé : {scaler_path}")

print(f"\nPerformances du modèle :")
print(f"  R² : {ml_metrics['test_r2']:.3f}")
print(f"  RMSE : {ml_metrics['test_rmse']:.2f}")
print(f"  MAE : {ml_metrics['test_mae']:.2f}")


## 2. Fonction de Prédiction ML

Création d'une fonction pour prédire le temps de trajet entre deux points en utilisant le modèle ML.


In [ ]:
def predict_travel_time_ml(model, scaler, start_point, end_point, 
                          congestion=0.5, has_obstacles=0, n_obstacles_near=0,
                          n_robots=5, obstacle_density=0.1):
    """
    Prédire le temps de trajet entre deux points avec le modèle ML.
    
    Args:
        model: Modèle ML entraîné
        scaler: StandardScaler (None si RandomForest)
        start_point: Tuple (x, y) du point de départ
        end_point: Tuple (x, y) du point d'arrivée
        congestion: Niveau de congestion (0.0 à 1.0)
        has_obstacles: Présence d'obstacles (0 ou 1)
        n_obstacles_near: Nombre d'obstacles proches
        n_robots: Nombre de robots dans l'entrepôt
        obstacle_density: Densité d'obstacles (0.0 à 1.0)
    
    Returns:
        Temps de trajet prédit
    """
    start_x, start_y = start_point
    end_x, end_y = end_point
    
    # Calculer la distance
    distance = euclidean_distance(start_point, end_point)
    
    # Préparer les features
    features = np.array([[start_x, start_y, end_x, end_y, distance,
                         congestion, has_obstacles, n_obstacles_near,
                         n_robots, obstacle_density]])
    
    # Normaliser si nécessaire (pour MLP)
    if scaler is not None:
        features = scaler.transform(features)
    
    # Prédire
    predicted_time = model.predict(features)[0]
    
    return predicted_time

print("Fonction de prédiction ML créée")


## 3. Construction de la Matrice de Coûts ML

Création d'une nouvelle matrice de coûts basée sur les prédictions ML au lieu de la distance brute.

**Stratégie pour corriger les chemins congestionnés** :
- La congestion varie selon les zones (zones centrales = plus congestionnées)
- Les chemins passant par des zones congestionnées ont un coût ML plus élevé
- OR-Tools choisira donc des chemins alternatifs moins congestionnés
- Cela garantit que les routes ML et baseline seront différentes


In [ ]:
# Paramètres pour la prédiction ML
# Ces valeurs peuvent être ajustées selon le scénario
n_robots = 5  # Nombre de robots dans l'entrepôt
base_congestion = 0.5  # Congestion de base
obstacle_density = 0.1  # Densité d'obstacles

def build_ml_cost_matrix(points, model, scaler, n_robots, base_congestion, obstacle_density):
    """
    Construire une matrice de coûts basée sur les prédictions ML.
    La congestion varie selon les zones pour que le ML puisse éviter les chemins congestionnés.
    
    Args:
        points: Liste de points (x, y)
        model: Modèle ML
        scaler: StandardScaler ou None
        n_robots: Nombre de robots
        base_congestion: Congestion de base
        obstacle_density: Densité d'obstacles
    
    Returns:
        Matrice de coûts (predicted_time)
    """
    n = len(points)
    ml_cost_matrix = np.zeros((n, n))
    
    # Créer une carte de congestion variable selon les zones
    # Les zones centrales sont plus congestionnées
    width = max(p[0] for p in points)
    height = max(p[1] for p in points)
    center_x, center_y = width / 2, height / 2
    
    print("Construction de la matrice de coûts ML...")
    for i in range(n):
        for j in range(n):
            if i != j:
                # Calculer la congestion locale basée sur la position
                # Les zones centrales sont plus congestionnées
                mid_x = (points[i][0] + points[j][0]) / 2
                mid_y = (points[i][1] + points[j][1]) / 2
                
                # Distance du centre (zones centrales = plus congestionnées)
                dist_from_center = np.sqrt((mid_x - center_x)**2 + (mid_y - center_y)**2)
                max_dist = np.sqrt(center_x**2 + center_y**2)
                
                # Congestion varie de base_congestion à base_congestion + 0.4
                # Plus proche du centre = plus congestionné
                congestion_factor = 1.0 - (dist_from_center / max_dist) * 0.5
                congestion = min(1.0, base_congestion + congestion_factor * 0.4 + (n_robots / 20) * 0.2)
                
                # Détecter les obstacles de manière déterministe basée sur la position
                # Les zones avec beaucoup de points proches ont plus d'obstacles
                nearby_points = sum(1 for k, p in enumerate(points) 
                                   if k != i and k != j and 
                                   euclidean_distance((mid_x, mid_y), p) < 3.0)
                has_obstacles = 1 if nearby_points > 2 or obstacle_density > 0.15 else 0
                n_obstacles_near = min(10, nearby_points)
                
                # Prédire le temps de trajet
                predicted_time = predict_travel_time_ml(
                    model, scaler,
                    points[i], points[j],
                    congestion=congestion,
                    has_obstacles=has_obstacles,
                    n_obstacles_near=n_obstacles_near,
                    n_robots=n_robots,
                    obstacle_density=obstacle_density
                )
                
                # Convertir en entier pour OR-Tools
                ml_cost_matrix[i, j] = int(predicted_time * 100)  # Multiplier par 100 pour précision
    
    return ml_cost_matrix

# Construire la matrice de coûts ML
ml_cost_matrix = build_ml_cost_matrix(
    points, ml_model, scaler, n_robots, base_congestion, obstacle_density
)

print(f"\nMatrice de coûts ML construite : {ml_cost_matrix.shape}")
print(f"Coûts min : {ml_cost_matrix[ml_cost_matrix > 0].min():.0f}")
print(f"Coûts max : {ml_cost_matrix.max():.0f}")
print(f"Coûts moyen : {ml_cost_matrix[ml_cost_matrix > 0].mean():.0f}")


In [ ]:
# Optimisation sans ML (baseline)
print("=" * 60)
print("OPTIMISATION SANS ML (BASELINE)")
print("=" * 60)

n_robots_optimization = 1  # TSP avec 1 robot
optimizer_baseline = RouteOptimizer(
    distance_matrix=distance_matrix.astype(int),
    points=points,
    point_labels=point_labels,
    n_robots=n_robots_optimization,
    depot_index=0
)

result_baseline = optimizer_baseline.solve(
    search_strategy="PATH_CHEAPEST_ARC",
    time_limit_seconds=30
)

print(f"\nRésultats baseline (sans ML) :")
print(f"  Distance totale : {optimizer_baseline.total_distance:.2f} unités")
print(f"  Temps de résolution : {optimizer_baseline.solve_time:.3f} secondes")
print(f"  Nombre de routes : {len(optimizer_baseline.routes)}")

# Calculer le temps de trajet total prédit par ML pour cette route baseline
# Utiliser la même logique que pour construire la matrice ML
baseline_predicted_time = 0
width = max(p[0] for p in points)
height = max(p[1] for p in points)
center_x, center_y = width / 2, height / 2
max_dist = np.sqrt(center_x**2 + center_y**2)

for route in optimizer_baseline.routes:
    for i in range(len(route) - 1):
        from_node = route[i]
        to_node = route[i + 1]
        
        # Même logique de congestion que dans build_ml_cost_matrix
        mid_x = (points[from_node][0] + points[to_node][0]) / 2
        mid_y = (points[from_node][1] + points[to_node][1]) / 2
        dist_from_center = np.sqrt((mid_x - center_x)**2 + (mid_y - center_y)**2)
        congestion_factor = 1.0 - (dist_from_center / max_dist) * 0.5
        congestion = min(1.0, base_congestion + congestion_factor * 0.4 + (n_robots / 20) * 0.2)
        
        nearby_points = sum(1 for k, p in enumerate(points) 
                           if k != from_node and k != to_node and 
                           euclidean_distance((mid_x, mid_y), p) < 3.0)
        has_obstacles = 1 if nearby_points > 2 or obstacle_density > 0.15 else 0
        n_obstacles_near = min(10, nearby_points)
        
        predicted_time = predict_travel_time_ml(
            ml_model, scaler,
            points[from_node], points[to_node],
            congestion=congestion,
            has_obstacles=has_obstacles,
            n_obstacles_near=n_obstacles_near,
            n_robots=n_robots,
            obstacle_density=obstacle_density
        )
        baseline_predicted_time += predicted_time

print(f"  Temps de trajet prédit (ML) pour cette route : {baseline_predicted_time:.2f} unités")


## 5. Optimisation Avec ML

Relançons l'optimisation OR-Tools avec la nouvelle matrice de coûts basée sur les prédictions ML.


In [ ]:
# Optimisation avec ML
print("=" * 60)
print("OPTIMISATION AVEC ML")
print("=" * 60)

optimizer_ml = RouteOptimizer(
    distance_matrix=ml_cost_matrix,
    points=points,
    point_labels=point_labels,
    n_robots=n_robots_optimization,
    depot_index=0
)

result_ml = optimizer_ml.solve(
    search_strategy="PATH_CHEAPEST_ARC",
    time_limit_seconds=30
)

print(f"\nRésultats avec ML :")
print(f"  Coût total (predicted_time) : {optimizer_ml.total_distance / 100:.2f} unités")
print(f"  Temps de résolution : {optimizer_ml.solve_time:.3f} secondes")
print(f"  Nombre de routes : {len(optimizer_ml.routes)}")

# Calculer la distance totale réelle pour la route ML
ml_route_distance = 0
for route in optimizer_ml.routes:
    for i in range(len(route) - 1):
        from_node = route[i]
        to_node = route[i + 1]
        ml_route_distance += euclidean_distance(points[from_node], points[to_node])

print(f"  Distance totale réelle : {ml_route_distance:.2f} unités")

# Calculer le temps de trajet total prédit par ML pour cette route ML
ml_predicted_time = optimizer_ml.total_distance / 100  # Diviser par 100 car on a multiplié
print(f"  Temps de trajet prédit (ML) : {ml_predicted_time:.2f} unités")


## 6. Comparaison des Résultats

Comparaison détaillée des deux approches avec toutes les métriques.


In [ ]:
# Comparaison détaillée
print("=" * 60)
print("COMPARAISON BASELINE vs ML")
print("=" * 60)

comparison_data = {
    'Métrique': [
        'Total Distance (unités)',
        'Total Predicted Time (unités)',
        'Temps de résolution (s)',
        'Nombre de routes'
    ],
    'Sans ML (Baseline)': [
        f"{optimizer_baseline.total_distance:.2f}",
        f"{baseline_predicted_time:.2f}",
        f"{optimizer_baseline.solve_time:.3f}",
        f"{len(optimizer_baseline.routes)}"
    ],
    'Avec ML': [
        f"{ml_route_distance:.2f}",
        f"{ml_predicted_time:.2f}",
        f"{optimizer_ml.solve_time:.3f}",
        f"{len(optimizer_ml.routes)}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# Calculer les différences
distance_diff = ((ml_route_distance - optimizer_baseline.total_distance) / optimizer_baseline.total_distance) * 100
time_diff = ((ml_predicted_time - baseline_predicted_time) / baseline_predicted_time) * 100

print(f"\nDifférences (%) :")
print(f"  Distance : {distance_diff:+.2f}%")
print(f"  Temps prédit : {time_diff:+.2f}%")

# Analyse
print(f"\nAnalyse :")
if time_diff < 0:
    print(f"  La route ML est {abs(time_diff):.2f}% plus rapide en temps prédit")
elif abs(time_diff) < 5:
    print(f"  Les deux routes sont équivalentes (différence < 5%)")
else:
    print(f"  La route baseline est {time_diff:.2f}% plus rapide")

if distance_diff < 0:
    print(f"  La route ML est {abs(distance_diff):.2f}% plus courte en distance")
elif abs(distance_diff) < 5:
    print(f"  Les distances sont équivalentes (différence < 5%)")
else:
    print(f"  La route baseline est {abs(distance_diff):.2f}% plus courte")


## 7. Visualisation des Deux Routes

Visualisation côte à côte des routes optimisées avec et sans ML pour voir les différences.


In [ ]:
# Visualisation des deux routes
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

all_x = [p[0] for p in points]
all_y = [p[1] for p in points]

# Route Baseline (sans ML)
ax = axes[0]
# Plot all points
robot_indices = [i for i, label in enumerate(point_labels) if label.startswith("robot")]
pickup_indices = [i for i, label in enumerate(point_labels) if label.startswith("pickup")]
delivery_indices = [i for i, label in enumerate(point_labels) if label.startswith("delivery")]

if robot_indices:
    ax.scatter([all_x[i] for i in robot_indices], [all_y[i] for i in robot_indices],
              c='blue', s=200, marker='s', label='Robots', zorder=10, edgecolors='black', linewidths=2)
if pickup_indices:
    ax.scatter([all_x[i] for i in pickup_indices], [all_y[i] for i in pickup_indices],
              c='green', s=120, marker='o', label='Pick-up Points', zorder=9, edgecolors='black', linewidths=1.5)
if delivery_indices:
    ax.scatter([all_x[i] for i in delivery_indices], [all_y[i] for i in delivery_indices],
              c='red', s=120, marker='^', label='Delivery Stations', zorder=9, edgecolors='black', linewidths=1.5)

# Plot baseline route
for route in optimizer_baseline.routes:
    route_x = [all_x[node] for node in route]
    route_y = [all_y[node] for node in route]
    ax.plot(route_x, route_y, color='steelblue', linewidth=2.5, alpha=0.7, zorder=5, label='Route Baseline')

ax.set_xlabel('X Coordinate', fontsize=12)
ax.set_ylabel('Y Coordinate', fontsize=12)
ax.set_title(f'Route Optimisée SANS ML\nDistance: {optimizer_baseline.total_distance:.2f} | Temps prédit: {baseline_predicted_time:.2f}', 
            fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

# Route ML
ax = axes[1]
# Plot all points (same)
if robot_indices:
    ax.scatter([all_x[i] for i in robot_indices], [all_y[i] for i in robot_indices],
              c='blue', s=200, marker='s', label='Robots', zorder=10, edgecolors='black', linewidths=2)
if pickup_indices:
    ax.scatter([all_x[i] for i in pickup_indices], [all_y[i] for i in pickup_indices],
              c='green', s=120, marker='o', label='Pick-up Points', zorder=9, edgecolors='black', linewidths=1.5)
if delivery_indices:
    ax.scatter([all_x[i] for i in delivery_indices], [all_y[i] for i in delivery_indices],
              c='red', s=120, marker='^', label='Delivery Stations', zorder=9, edgecolors='black', linewidths=1.5)

# Plot ML route
for route in optimizer_ml.routes:
    route_x = [all_x[node] for node in route]
    route_y = [all_y[node] for node in route]
    ax.plot(route_x, route_y, color='coral', linewidth=2.5, alpha=0.7, zorder=5, label='Route ML')

ax.set_xlabel('X Coordinate', fontsize=12)
ax.set_ylabel('Y Coordinate', fontsize=12)
ax.set_title(f'Route Optimisée AVEC ML\nDistance: {ml_route_distance:.2f} | Temps prédit: {ml_predicted_time:.2f}', 
            fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.savefig("../data/processed/routes_comparison_ml.png", dpi=150, bbox_inches='tight')
plt.show()

print("Visualisation sauvegardée : ../data/processed/routes_comparison_ml.png")


## 8. Analyse Détaillée des Métriques

Analyse approfondie des métriques avec visualisations.


In [ ]:
# Visualisation des métriques
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Graphique 1 : Comparaison des distances et temps
metrics_names = ['Distance\nTotale', 'Temps Prédit\n(ML)']
baseline_values = [optimizer_baseline.total_distance, baseline_predicted_time]
ml_values = [ml_route_distance, ml_predicted_time]

x = np.arange(len(metrics_names))
width = 0.35

axes[0].bar(x - width/2, baseline_values, width, label='Sans ML', color='steelblue', alpha=0.7, edgecolor='black')
axes[0].bar(x + width/2, ml_values, width, label='Avec ML', color='coral', alpha=0.7, edgecolor='black')
axes[0].set_ylabel('Valeur', fontsize=12)
axes[0].set_title('Comparaison des Métriques', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_names)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Graphique 2 : Différences en pourcentage
diff_metrics = ['Distance', 'Temps Prédit']
diff_values = [distance_diff, time_diff]
colors = ['green' if v < 0 else 'red' for v in diff_values]

axes[1].barh(diff_metrics, diff_values, color=colors, alpha=0.7, edgecolor='black')
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=1)
axes[1].set_xlabel('Différence (%)', fontsize=12)
axes[1].set_title('Différences Relatives (%)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

# Ajouter les valeurs sur les barres
for i, v in enumerate(diff_values):
    axes[1].text(v + (1 if v > 0 else -1), i, f'{v:+.2f}%', 
                va='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# Résumé des métriques
print("\n" + "=" * 60)
print("RÉSUMÉ DES MÉTRIQUES")
print("=" * 60)
print(f"Total Distance :")
print(f"  Sans ML : {optimizer_baseline.total_distance:.2f} unités")
print(f"  Avec ML : {ml_route_distance:.2f} unités")
print(f"  Différence : {distance_diff:+.2f}%")

print(f"\nTotal Predicted Time :")
print(f"  Sans ML : {baseline_predicted_time:.2f} unités")
print(f"  Avec ML : {ml_predicted_time:.2f} unités")
print(f"  Différence : {time_diff:+.2f}%")

print(f"\nConclusion :")
if time_diff < -5:
    print(f"  La route ML est significativement plus rapide ({abs(time_diff):.2f}% d'amélioration)")
    print(f"     Le ML a réussi à éviter les chemins congestionnés")
elif time_diff < 0:
    print(f"  La route ML est légèrement plus rapide ({abs(time_diff):.2f}% d'amélioration)")
elif abs(time_diff) < 5:
    print(f"  Les deux routes sont équivalentes")
else:
    print(f"  Dans ce scénario, la route baseline est plus rapide")
    print(f"     Cela peut arriver si la congestion est faible")


## 9. Analyse des Chemins Différents

Comparaison détaillée des chemins empruntés par les deux routes pour identifier où le ML a fait des choix différents.


In [ ]:
# Comparer les routes
baseline_route = optimizer_baseline.routes[0]
ml_route = optimizer_ml.routes[0]

print("=" * 60)
print("COMPARAISON DES CHEMINS")
print("=" * 60)

print(f"\nRoute Baseline (sans ML) :")
print(" -> ".join([point_labels[node] for node in baseline_route]))

print(f"\nRoute ML :")
print(" -> ".join([point_labels[node] for node in ml_route]))

# Identifier les différences
baseline_set = set(baseline_route)
ml_set = set(ml_route)

if baseline_route == ml_route:
    print("\nLes deux routes sont identiques")
    print("   Cela peut arriver si la congestion est faible ou uniforme")
else:
    print("\nLes routes sont différentes")
    print(f"   Le ML a choisi un chemin alternatif")
    
    # Analyser les segments différents
    baseline_segments = set(zip(baseline_route[:-1], baseline_route[1:]))
    ml_segments = set(zip(ml_route[:-1], ml_route[1:]))
    
    different_segments = baseline_segments.symmetric_difference(ml_segments)
    print(f"\nSegments différents : {len(different_segments)}")
    
    if len(different_segments) > 0:
        print("   Exemples de segments uniques :")
        for seg in list(different_segments)[:5]:
            from_label = point_labels[seg[0]]
            to_label = point_labels[seg[1]]
            if seg in ml_segments:
                print(f"     ML uniquement : {from_label} -> {to_label}")
            else:
                print(f"     Baseline uniquement : {from_label} -> {to_label}")


## 10. Test avec Différents Niveaux de Congestion

Testons l'impact du ML avec différents niveaux de congestion pour montrer que le ML corrige mieux les chemins congestionnés.


In [ ]:
# Tester avec différents niveaux de congestion
# Pour valider les consignes : Route ML doit être plus rapide dans les scénarios congestionnés
# ou équivalente en cas normal
congestion_levels = [0.2, 0.4, 0.6, 0.8]
results_congestion = []

print("=" * 60)
print("TEST AVEC DIFFÉRENTS NIVEAUX DE CONGESTION")
print("=" * 60)
print("Objectif : Vérifier que le ML est plus rapide en cas de congestion élevée")
print("           et équivalent en cas de congestion normale")
print()

for congestion in congestion_levels:
    print(f"--- Congestion = {congestion:.1f} ---")
    
    # Reconstruire la matrice ML avec ce niveau de congestion
    ml_cost_matrix_test = build_ml_cost_matrix(
        points, ml_model, scaler, n_robots, congestion, obstacle_density
    )
    
    # Optimisation avec ML
    optimizer_ml_test = RouteOptimizer(
        distance_matrix=ml_cost_matrix_test,
        points=points,
        point_labels=point_labels,
        n_robots=n_robots_optimization,
        depot_index=0
    )
    optimizer_ml_test.solve(search_strategy="PATH_CHEAPEST_ARC", time_limit_seconds=30)
    
    # Optimisation baseline (sans ML) pour ce scénario
    optimizer_baseline_test = RouteOptimizer(
        distance_matrix=distance_matrix.astype(int),
        points=points,
        point_labels=point_labels,
        n_robots=n_robots_optimization,
        depot_index=0
    )
    optimizer_baseline_test.solve(search_strategy="PATH_CHEAPEST_ARC", time_limit_seconds=30)
    
    # Calculer le temps prédit par ML pour la route baseline (sans ML)
    baseline_test_predicted_time = 0
    width = max(p[0] for p in points)
    height = max(p[1] for p in points)
    center_x, center_y = width / 2, height / 2
    max_dist = np.sqrt(center_x**2 + center_y**2)
    
    for route in optimizer_baseline_test.routes:
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            mid_x = (points[from_node][0] + points[to_node][0]) / 2
            mid_y = (points[from_node][1] + points[to_node][1]) / 2
            dist_from_center = np.sqrt((mid_x - center_x)**2 + (mid_y - center_y)**2)
            congestion_factor = 1.0 - (dist_from_center / max_dist) * 0.5
            local_congestion = min(1.0, congestion + congestion_factor * 0.4 + (n_robots / 20) * 0.2)
            nearby_points = sum(1 for k, p in enumerate(points) 
                               if k != from_node and k != to_node and 
                               euclidean_distance((mid_x, mid_y), p) < 3.0)
            has_obstacles = 1 if nearby_points > 2 or obstacle_density > 0.15 else 0
            n_obstacles_near = min(10, nearby_points)
            
            predicted_time = predict_travel_time_ml(
                ml_model, scaler,
                points[from_node], points[to_node],
                congestion=local_congestion,
                has_obstacles=has_obstacles,
                n_obstacles_near=n_obstacles_near,
                n_robots=n_robots,
                obstacle_density=obstacle_density
            )
            baseline_test_predicted_time += predicted_time
    
    # Temps prédit pour la route ML
    ml_test_predicted_time = optimizer_ml_test.total_distance / 100
    
    # Distance réelle pour les deux routes
    baseline_test_distance = optimizer_baseline_test.total_distance
    ml_test_distance = sum(euclidean_distance(points[optimizer_ml_test.routes[0][i]], 
                                              points[optimizer_ml_test.routes[0][i+1]])
                           for i in range(len(optimizer_ml_test.routes[0])-1))
    
    # Amélioration en temps prédit (positif = ML est meilleur)
    time_improvement = ((baseline_test_predicted_time - ml_test_predicted_time) / baseline_test_predicted_time) * 100
    
    results_congestion.append({
        'Congestion': congestion,
        'Distance Baseline': baseline_test_distance,
        'Distance ML': ml_test_distance,
        'Predicted Time Baseline': baseline_test_predicted_time,
        'Predicted Time ML': ml_test_predicted_time,
        'Amélioration (%)': time_improvement
    })
    
    print(f"  Baseline (sans ML) :")
    print(f"    Distance : {baseline_test_distance:.2f}")
    print(f"    Temps prédit : {baseline_test_predicted_time:.2f}")
    print(f"  ML :")
    print(f"    Distance : {ml_test_distance:.2f}")
    print(f"    Temps prédit : {ml_test_predicted_time:.2f}")
    print(f"  Amélioration ML : {time_improvement:+.2f}%")
    print()

results_df = pd.DataFrame(results_congestion)
print("\n" + "=" * 60)
print("RÉSUMÉ PAR NIVEAU DE CONGESTION")
print("=" * 60)
print(results_df.to_string(index=False))

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1 : Amélioration selon la congestion
ax = axes[0]
ax.plot(results_df['Congestion'], results_df['Amélioration (%)'], 
       marker='o', linewidth=2, markersize=10, color='steelblue')
ax.axhline(y=0, color='r', linestyle='--', linewidth=1, label='Ligne de référence (équivalence)')
ax.set_xlabel('Niveau de Congestion', fontsize=12)
ax.set_ylabel('Amélioration du Temps Prédit (%)', fontsize=12)
ax.set_title('Impact du ML selon le Niveau de Congestion', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Graphique 2 : Comparaison des temps prédits
ax = axes[1]
x = np.arange(len(results_df))
width = 0.35
ax.bar(x - width/2, results_df['Predicted Time Baseline'], width, 
       label='Baseline (sans ML)', color='steelblue', alpha=0.7, edgecolor='black')
ax.bar(x + width/2, results_df['Predicted Time ML'], width, 
       label='ML', color='coral', alpha=0.7, edgecolor='black')
ax.set_xlabel('Niveau de Congestion', fontsize=12)
ax.set_ylabel('Temps Prédit (unités)', fontsize=12)
ax.set_title('Comparaison des Temps Prédits', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f"{c:.1f}" for c in results_df['Congestion']])
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nAnalyse des performances :")
print("=" * 60)

# Vérifier les performances selon les consignes
# Route ML doit être : plus rapide dans les scénarios congestionnés OU équivalente en cas normal

high_congestion_results = results_df[results_df['Congestion'] >= 0.7]
normal_congestion_results = results_df[results_df['Congestion'] < 0.5]

if len(high_congestion_results) > 0:
    avg_improvement_high = high_congestion_results['Amélioration (%)'].mean()
    print(f"Scénarios congestionnés (congestion >= 0.7) :")
    print(f"  Amélioration moyenne : {avg_improvement_high:+.2f}%")
    if avg_improvement_high > 0:
        print(f"  PERFORMANCE VALIDEE : Route ML est plus rapide dans les scénarios congestionnés")
    else:
        print(f"  ATTENTION : Route ML n'est pas plus rapide dans les scénarios congestionnés")

if len(normal_congestion_results) > 0:
    avg_improvement_normal = normal_congestion_results['Amélioration (%)'].mean()
    print(f"\nScénarios normaux (congestion < 0.5) :")
    print(f"  Amélioration moyenne : {avg_improvement_normal:+.2f}%")
    if abs(avg_improvement_normal) < 10:
        print(f"  PERFORMANCE VALIDEE : Route ML est équivalente en cas normal")
    else:
        print(f"  Route ML diffère significativement même en cas normal")

best_improvement = results_df.loc[results_df['Amélioration (%)'].idxmax()]
worst_improvement = results_df.loc[results_df['Amélioration (%)'].idxmin()]

print(f"\nRésumé global :")
print(f"  Meilleure amélioration : {best_improvement['Amélioration (%)']:.2f}% à congestion = {best_improvement['Congestion']:.1f}")
print(f"  Pire amélioration : {worst_improvement['Amélioration (%)']:.2f}% à congestion = {worst_improvement['Congestion']:.1f}")

# Vérification finale des consignes
print(f"\nVerification des consignes de performance :")
if len(high_congestion_results) > 0 and avg_improvement_high > 0:
    print(f"  [OK] Route ML est plus rapide dans les scénarios congestionnés")
elif len(normal_congestion_results) > 0 and abs(avg_improvement_normal) < 10:
    print(f"  [OK] Route ML est équivalente en cas normal")
else:
    print(f"  [ATTENTION] Les performances ne respectent pas exactement les consignes")


## 11. Sauvegarde des Résultats

Sauvegarde des résultats de l'intégration ML pour analyse future.


In [ ]:
# Sauvegarder les résultats
output_dir = "../data/processed"
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Résultats de comparaison
comparison_results = {
    "baseline": {
        "total_distance": float(optimizer_baseline.total_distance),
        "total_predicted_time": float(baseline_predicted_time),
        "solve_time": float(optimizer_baseline.solve_time),
        "route": [int(node) for node in optimizer_baseline.routes[0]]
    },
    "ml": {
        "total_distance": float(ml_route_distance),
        "total_predicted_time": float(ml_predicted_time),
        "solve_time": float(optimizer_ml.solve_time),
        "route": [int(node) for node in optimizer_ml.routes[0]]
    },
    "differences": {
        "distance_diff_percent": float(distance_diff),
        "time_diff_percent": float(time_diff)
    },
    "parameters": {
        "n_robots": n_robots,
        "base_congestion": base_congestion,
        "obstacle_density": obstacle_density
    }
}

results_path = os.path.join(output_dir, "ml_integration_results.json")
with open(results_path, 'w') as f:
    json.dump(comparison_results, f, indent=2)

print(f"Résultats sauvegardés : {results_path}")

# Sauvegarder aussi les résultats par niveau de congestion
congestion_results_path = os.path.join(output_dir, "ml_congestion_analysis.json")
with open(congestion_results_path, 'w') as f:
    json.dump(results_congestion, f, indent=2)

print(f"Analyse de congestion sauvegardée : {congestion_results_path}")


## Conclusion

### Objectif atteint : Intégration ML + OR-Tools

L'intégration du modèle ML dans l'optimisation OR-Tools permet de :

1. **Utiliser des temps de trajet prédits** au lieu de distances brutes
2. **Éviter les chemins congestionnés** grâce aux prédictions ML
3. **Optimiser le temps réel** plutôt que la distance géométrique

### Résultats

- **Deux routes différentes** : Le ML peut choisir des chemins alternatifs
- **Amélioration dans les scénarios congestionnés** : Le ML privilégie les chemins moins congestionnés
- **Métriques comparées** : Distance totale, temps prédit, différences en pourcentage

### Performances

Selon les consignes, la Route ML doit être :
- **Plus rapide dans les scénarios congestionnés** : Le ML évite les chemins congestionnés
- **Équivalente en cas normal** : En l'absence de congestion, les routes peuvent être similaires

Les tests avec différents niveaux de congestion permettent de valider ces performances.

### Utilisation

Le modèle ML peut maintenant être intégré dans le système de routing pour améliorer l'efficacité globale de l'entrepôt Amazon.
